In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# load data
file_name = 'sales.xlsx'
df = pd.read_excel("data/"+file_name)
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2,4,Credit Card,Takeaway,2023-09-08 00:00:00
1,TXN_4977031,Cake,4,3,12,Cash,In-store,2023-05-16 00:00:00
2,TXN_4271903,Cookie,4,1,ERROR,Credit Card,In-store,2023-07-19 00:00:00
3,TXN_7034554,Salad,2,5,10,UNKNOWN,UNKNOWN,2023-04-27 00:00:00
4,TXN_3160411,Coffee,2,2,4,Digital Wallet,In-store,2023-06-11 00:00:00
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2,4,NaN,UNKNOWN,2023-08-30 00:00:00
9996,TXN_9659401,NaN,3,NaN,3,Digital Wallet,NaN,2023-06-02 00:00:00
9997,TXN_5255387,Coffee,4,2,8,Digital Wallet,NaN,2023-03-02 00:00:00
9998,TXN_7695629,Cookie,3,NaN,3,Digital Wallet,NaN,2023-12-02 00:00:00


In [3]:
print(len("                                                                               "))

79


In [4]:
df = df.drop_duplicates()

def validate_ID(df, column):
    """
    Validates and cleans a column of type ID:
    - Removes rows with null IDs
    - Removes duplicate rows
    - Validates the format of the IDs and removes the invalid ones
    - Logs the summary of changes made

    Parameters:
        df (DataFrame): The input DataFrame
        column (str): The name of the ID column to validate

    Returns:
        DataFrame: Updated DataFrame with cleaned ID values
    """
    
    validation_log = {
    'column': column,
    'nulls_removed': 0,
    'duplicates_removed': 0,
    'invalid_ids_removed': 0,
    'invalid_samples': []
    }

    # Handle NULLs
    null_mask = df[column].isna()
    if null_mask.any():
        validation_log['nulls_removed'] = null_mask.sum()
        df = df[~null_mask].copy()

    # Check for duplicates
    duplicate_mask = df.duplicated(subset=column, keep="first")
    if duplicate_mask.any():
        validation_log["duplicates_removed"] = duplicate_mask.sum()
        df = df[~duplicate_mask].copy()

    # Validate the format of IDs
    valid_mask = df[column].str.match(r'^TXN_\d{7}$', case=False, na=False)
    invalid_mask = ~valid_mask
    if invalid_mask.any():
        validation_log["invalid_ids_removed"] = invalid_mask.sum()
        validation_log['invalid_samples'] = df.loc[
            invalid_mask, column
        ].head(3).tolist()
        df = df[valid_mask].copy()

    # Print summary
    print(f"\n--- Validation Report: {column} ---")
    print(f"NULLs removed: {validation_log['nulls_removed']}")
    print(f"Duplicates removed: {validation_log['duplicates_removed']}")
    print(f"Invalid IDs removed: {validation_log['invalid_ids_removed']}")
    if validation_log['invalid_ids_removed'] > 0:
        print(f"Sample invalid IDs: {validation_log['invalid_samples']}")
        print(f"Expected format: 'TXN_1234567' (case-insensitive).")

    return df
                                                                               ##########################
def validate_categorical(df, column, values):
    """
    Validates and cleans a categorical column:
    - Replaces nulls with 'unknown'
    - Replaces invalid text with unknown
    - Prints the summary of changes made

    Parameters:
        df (DataFrame): The input DataFrame
        column (str): The name of the ID column to validate
        values (str list): The names of allowed categorical values

    Returns:
        DataFrame: Updated DataFrame with cleaned categorical values
    """
    
    df[column] = df[column].str.lower()

    validation_log = {
    'column': column,
    'nulls_replaced': 0,
    'invalid_text_replaced': 0,
    'invalid_samples': []
    }

    # Handle NULLs
    null_mask = df[column].isna()
    if null_mask.any():
        validation_log['nulls_replaced'] = null_mask.sum()
        df[column] = df[column].fillna('unknown')

    # Handle invalid text
    valid_mask = df[column].isin(values)
    invalid_mask = ~valid_mask
    invalid_rows = invalid_mask.copy()
    validation_log['invalid_text_replaced'] = invalid_mask.sum()
    validation_log['invalid_samples'] = df.loc[
        invalid_mask, column
    ].head(3).tolist()
    df[column] = df[column].where(~invalid_mask, 'unknown')
    
    # Print summary
    print(f"\n--- Validation Report: {column} ---")
    print(f"NULLs replaced: {validation_log['nulls_replaced']}")
    print(f"Invalid text replaced: {validation_log['invalid_text_replaced']}")
    if validation_log['invalid_text_replaced'] > 0:
        print(f"Sample invalid text: {validation_log['invalid_samples']}")

    return df

def validate_numeric(df, column):
    """
    Validates and cleans a numeric column:
    - Replaces invalid text with null
    - Prints the summary of changes made

    Parameters:
        df (DataFrame): The input DataFrame
        column (str): The name of the numeric column to validate

    Returns:
        DataFrame: Updated DataFrame with cleaned numeric values
    """
    
    validation_log = {
    'column': column,
    'original_nulls': 0,
    'invalid_strings': 0, 
    'invalid_samples': [],
    'final_total_nulls': 0
    }

    # Handle nulls
    original_nulls = df[column].isna()
    validation_log['original_nulls'] = original_nulls.sum()
    invalid_mask = pd.to_numeric(df[column], errors='coerce').isna()

    # Handle invalid values
    invalid_string_mask = invalid_mask & ~original_nulls
    validation_log['invalid_strings'] = invalid_string_mask.sum()
    validation_log['invalid_samples'] = df.loc[
        invalid_string_mask, column
    ].tolist()[:3]
    df[column] = pd.to_numeric(df[column], errors="coerce")
    validation_log['final_total_nulls'] = df[column].isna().sum()

    # Print summary
    print(f"\n--- Validation Report: {column} ---")
    print(f"Original nulls: {validation_log['original_nulls']}")
    print(f"Invalid strings: {validation_log['invalid_strings']}")
    if validation_log['invalid_strings'] > 0:
        print(f"Sample invalid text: {validation_log['invalid_samples']}")
    print(f"Final Total Nulls: {validation_log['final_total_nulls']}")

    return df

def validate_datetime(df, column):

    """
    Validates and cleans a datetime column:
    - Replaces invalid dates with null
    - Prints the summary of changes made

    Parameters:
        df (DataFrame): The input DataFrame
        column (str): The name of the datetime column to validate

    Returns:
        DataFrame: Updated DataFrame with cleaned datetime values
    """

    validation_log = {
    'column': column,
    'original_nulls': 0,
    'invalid_dates': 0, 
    'invalid_samples': [],
    'final_total_nulls': 0
    }

    # Handle nulls
    original_nulls = df[column].isna()
    validation_log['original_nulls'] = original_nulls.sum()
    invalid_mask = pd.to_datetime(df[column], errors='coerce').isna()

    # Handle invalid dates
    invalid_mask_no_null = invalid_mask & ~original_nulls
    validation_log['invalid_dates'] = invalid_mask_no_null.sum()
    validation_log['invalid_samples'] = df.loc[
        invalid_mask_no_null, column].tolist()[:3]
    df[column] = pd.to_datetime(df[column], errors="coerce")
    validation_log['final_total_nulls'] = df[column].isna().sum()

    # Print summary
    print(f"\n--- Validation Report: {column} ---")
    print(f"Original nulls: {validation_log['original_nulls']}")
    print(f"Invalid dates: {validation_log['invalid_dates']}")
    if validation_log['invalid_dates'] > 0:
        print(f"Sample invalid dates: {validation_log['invalid_samples']}")
    print(f"Final Total Nulls: {validation_log['final_total_nulls']}")

    return df

def validate_derived_column(df, derived_col, input_cols, formula_fn):
    
    """
    Validates and replaces the values of a derived column
    - Replaces invalid values with the correct values derived
    from input columns.
    - Prints the summary of changes made

    Parameters:
        df (DataFrame): The input DataFrame
        derived_col (str): The name of the derived column to validate
        input_cols (str list): The names of the columns from which the
        derived column is calculated
        formula_fn: A function that performs the calculation on the
        input columns to get the expected derived column and returns
        it as a series.

    Returns:
        DataFrame: Updated DataFrame with cleaned derived column values
    """
    
    expected = formula_fn(df)
    invalid_mask = df[derived_col] != expected

    validation_log = {
        'column': derived_col,
        'invalid_rows': invalid_mask.sum(),
        'invalid_samples': df[invalid_mask][input_cols + [derived_col]].head(3)
    }

    df.loc[invalid_mask, derived_col] = expected[invalid_mask]

    # Print report
    print(f"\n--- Derived Column Validation (Exact Match): {derived_col} ---")
    print(f"Invalid rows found: {validation_log['invalid_rows']}")
    if validation_log['invalid_rows'] > 0:
        print(f"Sample invalid values:\n {validation_log['invalid_samples']}")

    return df
        



df = validate_ID(df, 'Transaction ID')

item_values = [
    'juice', 'coffee', 'salad',
    'cake', 'sandwich', 'smoothie',
    'cookie', 'tea', 'unknown'
]
df = validate_categorical(df, 'Item', item_values)

df = validate_numeric(df, 'Quantity')

df = validate_numeric(df, 'Price Per Unit')

df = validate_numeric(df, 'Total Spent')

payment_values = [
    'digital wallet', 'credit card',
    'cash', 'unknown'
]

df = validate_categorical(df, 'Payment Method', payment_values)

location_values = [
    'takeaway', 'in-store',
    'unknown']

df = validate_categorical(df, 'Location', location_values)

df = validate_datetime(df, 'Transaction Date')

def compute_total_spent(df):
    return df['Quantity'] * df['Price Per Unit']

df = validate_derived_column(
    df,
    'Total Spent',
    ['Quantity', 'Price Per Unit'],
    compute_total_spent
)



--- Validation Report: Transaction ID ---
NULLs removed: 0
Duplicates removed: 0
Invalid IDs removed: 0

--- Validation Report: Item ---
NULLs replaced: 333
Invalid text replaced: 292
Sample invalid text: ['error', 'error', 'error']

--- Validation Report: Quantity ---
Original nulls: 138
Invalid strings: 341
Sample invalid text: ['ERROR', 'ERROR', 'UNKNOWN']
Final Total Nulls: 479

--- Validation Report: Price Per Unit ---
Original nulls: 179
Invalid strings: 354
Sample invalid text: ['ERROR', 'UNKNOWN', 'UNKNOWN']
Final Total Nulls: 533

--- Validation Report: Total Spent ---
Original nulls: 173
Invalid strings: 329
Sample invalid text: ['ERROR', 'ERROR', 'UNKNOWN']
Final Total Nulls: 502

--- Validation Report: Payment Method ---
NULLs replaced: 2579
Invalid text replaced: 306
Sample invalid text: ['error', 'error', 'error']

--- Validation Report: Location ---
NULLs replaced: 3265
Invalid text replaced: 358
Sample invalid text: ['error', 'error', 'error']

--- Validation Report: T

In [5]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"output/cleaned_sales_{timestamp}.xlsx"
df.to_excel(output_path, index=False)